[View in Colaboratory](https://colab.research.google.com/github/mayankprhr/handwritingrecog/blob/master/handwritingrecog.ipynb)

In [1]:
from sklearn.externals import joblib   #save claasifier in a file so that we can use it again without training it again
from sklearn import datasets 		   #download mnist package database 
from skimage.feature import hog		   #to calculate hog features(HISTOGRAM OF ORIENTED GAUSSIANS)
from sklearn.svm import LinearSVC 	   #to perform prediction after training
import numpy as np  				   #to store hog features in numpy arrays
from collections import Counter

dataset=datasets.fetch_mldata("MNIST Original")			#downlaod db
features=np.array(dataset.data, 'int16')
labels=np.array(dataset.target, 'int')

#calc hog features
list_hog_fd=[]				#initialise empty list where hog features are appended for each sample
for feature in features:
	#calc hog features and append them to list list_hog_fd	
	#set no. of cells in each individual cells to 1 of size 14x14 each
	#set orientation vector to 9, hog feature vector for each sample is 4x9=36  ????
	fd=hog(feature.reshape((28,28)),orientations=9, pixels_per_cell=(14,14),cells_per_block=(1,1),visualise=False)
	list_hog_fd.append(fd)

#create numpy array containing hog features to train classifier
hog_features=np.array(list_hog_fd,'float64')
print "Count of digits in dataset",Counter(labels)
#create linear SVM #multi class classifier is needed
clf=LinearSVC()
clf.fit(hog_features,labels)		#perform training using fit member function

#save the trained classifier in file
joblib.dump(clf,"digits_cls.pkl",compress=3) 			#to set how much compression is done more compression leads to slower read and writes





/usr/local/lib/python2.7/dist-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Count of digits in dataset Counter({1: 7877, 7: 7293, 3: 7141, 2: 6990, 9: 6958, 0: 6903, 6: 6876, 8: 6825, 4: 6824, 5: 6313})


['digits_cls.pkl']

In [1]:
!pip install tensorflow-gpu

    100% |████████████████████████████████| 258.9MB 73kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x56548bfd0000 @  0x7f406c9491c4 0x56543c0720d8 0x56543c15bd5d 0x56543c08577a 0x56543c08a462 0x56543c082b3a 0x56543c08a82e 0x56543c082b3a 0x56543c08a82e 0x56543c082b3a 0x56543c08a82e 0x56543c082b3a 0x56543c08ae1f 0x56543c082b3a 0x56543c08a82e 0x56543c082b3a 0x56543c08a82e 0x56543c08a462 0x56543c08a462 0x56543c082b3a 0x56543c08ae1f 0x56543c08a462 0x56543c082b3a 0x56543c08ae1f 0x56543c082b3a 0x56543c08ae1f 0x56543c082b3a 0x56543c08a82e 0x56543c082b3a 0x56543c0b350f 0x56543c0ae202


In [0]:
#load mods
import cv2
from sklearn.externals import joblib
from skimage.feature import hog
import numpy as np

#load classifier from file saved previously
clf=joblib.load("digits_cls.pkl")

#read input image #loading test image
im=cv2.imread("photo_1.jpg")

#convert to greyscale and apply gaussian filter
im_gray=cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)		#convert into grayscale img using cv2.cvtColor
im_gray=cv2.GaussianBlur(im_gray,(5,5),0)		#apply gaussian filter to remove noisy pixels

#threshold image
ret,im_th=cv2.threshold(im_gray,90,255, cv2.THRESH_BINARY_INV)  	#convert grayscale img to bimary img using thresold val=90
#all pixel locs with grayscale val>90 are set to 0 and with less than 90 are converted to 255 in binary img
#find contours(outline or boundary) in image
im,ctrs,hier=cv2.findContours(im_th.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#im2, contours, hierarchy = cv2.findContours(a, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#get rectangles contain each contour
rects=[cv2.boundingRect(ctr) for ctr in ctrs]  #calc bounding box for each contour


#for each rectangular region calc hog features and predict digit using linear SVM
for rect in rects:		#for each bounding box gen bounding box around each contour
	#draw rectangles
	cv2.rectangle(im, (rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3]),(0,255,0),3)
	#make rectangular region around digit
	leng=int(rect[3]*1.6)
	pt1=int(rect[1]+rect[3]//2-leng//2)
	pt2=int(rect[0]+rect[2]//2-leng//2)
	roi=im_th[pt1:pt1+leng,pt2:pt2+leng]
	#resize image
	roi=cv2.resize(roi,(28,28),interpolation=cv2.INTER_AREA) 	#resize each bounding square to 28x28
	roi=cv2.dilate(roi,(3,3))					#enlarge it 
	#calc the hog features 	#hog feature vector for each bounding square must be same as for classfier trained
	roi_hog_fd = hog(roi,orientations=9,pixels_per_cell=(14,14),cells_per_block=(1,1),visualise=False)
	nbr=clf.predict(np.array([roi_hog_fd],'float64'))	#predict digit using classifier 	
	cv2.putText(im, str(int(nbr[0])),(rect[0],rect[1]),cv2.FONT_HERSHEY_DUPLEX,2,(0,255,255),3)   	#draw bounding box around predicted digit

cv2.imshow("Resulting image with rectangular ROIs",im)  	#display image
cv2.waitKey()